##  Home Equity Loans

Train a classifer and save mode as a pickle file

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/data/hmeq.csv')

In [3]:
# Get Column Names so you can Start Designing
df.columns

Index(['BAD', 'LOAN', 'MORTDUE', 'VALUE', 'REASON', 'JOB', 'YOJ', 'DEROG',
       'DELINQ', 'CLAGE', 'NINQ', 'CLNO', 'DEBTINC'],
      dtype='object')

In [4]:
# Fill missing values in 'DEBTINC' with the median
#df['DEBTINC'] = df['DEBTINC'].fillna(df['DEBTINC'].median())


In [5]:
# Create a flag for missing 'DEBTINC' values; Its predictive for these data
#df['DEBTINC_missing'] = df['DEBTINC'].isnull().astype(int)

In [6]:
# Encode 'BAD' column to make it easier on the eye for our scoring app
df['BAD'] = df['BAD'].map({1: 'Yes', 0: 'No'})

In [7]:
# Create dummy variables for categorical features
dummy_cols = ['REASON', 'JOB']
df = pd.get_dummies(df, columns=dummy_cols, drop_first=True)

# Display the updated DataFrame
print(df.head())

   BAD  LOAN  MORTDUE     VALUE   YOJ  DEROG  DELINQ       CLAGE  NINQ  CLNO  \
0  Yes  1100  25860.0   39025.0  10.5    0.0     0.0   94.366667   1.0   9.0   
1  Yes  1300  70053.0   68400.0   7.0    0.0     2.0  121.833333   0.0  14.0   
2  Yes  1500  13500.0   16700.0   4.0    0.0     0.0  149.466667   1.0  10.0   
3  Yes  1500      NaN       NaN   NaN    NaN     NaN         NaN   NaN   NaN   
4   No  1700  97800.0  112000.0   3.0    0.0     0.0   93.333333   0.0  14.0   

   DEBTINC  REASON_HomeImp  JOB_Office  JOB_Other  JOB_ProfExe  JOB_Sales  \
0      NaN            True       False       True        False      False   
1      NaN            True       False       True        False      False   
2      NaN            True       False       True        False      False   
3      NaN           False       False      False        False      False   
4      NaN            True        True      False        False      False   

   JOB_Self  
0     False  
1     False  
2     False  


In [8]:
# Find bounds for data entry in app
print(df.describe())

               LOAN        MORTDUE          VALUE          YOJ        DEROG  \
count   5960.000000    5442.000000    5848.000000  5445.000000  5252.000000   
mean   18607.969799   73760.817200  101776.048741     8.922268     0.254570   
std    11207.480417   44457.609458   57385.775334     7.573982     0.846047   
min     1100.000000    2063.000000    8000.000000     0.000000     0.000000   
25%    11100.000000   46276.000000   66075.500000     3.000000     0.000000   
50%    16300.000000   65019.000000   89235.500000     7.000000     0.000000   
75%    23300.000000   91488.000000  119824.250000    13.000000     0.000000   
max    89900.000000  399550.000000  855909.000000    41.000000    10.000000   

            DELINQ        CLAGE         NINQ         CLNO      DEBTINC  
count  5380.000000  5652.000000  5450.000000  5738.000000  4693.000000  
mean      0.449442   179.766275     1.186055    21.296096    33.779915  
std       1.127266    85.810092     1.728675    10.138933     8.60174

In [9]:
# Train a decision tree

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# Define features (X) and target (y)
X = df.drop('BAD', axis=1)
y = df['BAD']

# Convert 'BAD' to numerical for the model
y = y.map({'Yes': 1, 'No': 0})

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


# Initialize a Decision Tree classifier
clf = DecisionTreeClassifier(random_state=42)

# Train the decision tree

from sklearn.metrics import accuracy_score, recall_score, f1_score, precision_score
clf.fit(X_train, y_train)
y_pred = clf.predict(X_val)
accuracy = accuracy_score(y_val,y_pred)
recall = recall_score(y_val, y_pred)
f1 = f1_score(y_val, y_pred)
precision = precision_score(y_val, y_pred)

# Print results
print(f'Accuracy: {accuracy}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')
print(f'Precision: {precision}')


Accuracy: 0.8833892617449665
Recall: 0.720754716981132
F1 Score: 0.7332053742802304
Precision: 0.74609375


In [ ]:
print(X_train)

       LOAN  MORTDUE     VALUE   YOJ  DEROG  DELINQ       CLAGE  NINQ  CLNO  \
944    9100  62976.0   77829.0   0.0    0.0     2.0  131.193496   0.0  23.0   
5755  43800   5884.0   64890.0   NaN    0.0     0.0  309.827548   0.0  19.0   
2259  13800  75142.0   95667.0  19.0    0.0     0.0  230.457554  10.0  24.0   
12     2000  45000.0   55000.0   3.0    0.0     0.0   86.066667   2.0  25.0   
1117   9900  47367.0  114837.0   4.0    0.0     0.0  210.435534   0.0  10.0   
...     ...      ...       ...   ...    ...     ...         ...   ...   ...   
3772  20000  99300.0  139150.0  10.0    0.0     0.0  162.366667   1.0  23.0   
5191  28000  49764.0   74526.0  24.0    NaN     1.0  294.594783   1.0  30.0   
5226  28300  71973.0  119224.0   4.0    0.0     0.0   89.379952   2.0  32.0   
5390  31100  73386.0  108864.0   5.0    NaN     NaN  119.539518  10.0  21.0   
860    8700  77930.0   80164.0  20.0    0.0     0.0  350.528410   1.0  47.0   

        DEBTINC  REASON_HomeImp  JOB_Office  JOB_Ot

In [10]:
import pickle


# Save the model to a pickle file named "hmeq_model.pkl"
with open("hmeq_model.pkl", "wb") as file:
    pickle.dump(clf, file)

"Model saved as hmeq_model.pkl"


'Model saved as hmeq_model.pkl'

In [11]:

feature_names = X_train.columns  # Replace X_train.columns with the list of feature names if needed

# Get feature importances from the decision tree
feature_importances = clf.feature_importances_

# Filter features with non-zero importance, indicating they were used in splits
used_features = feature_names[feature_importances > 0]

# Display the features used in the tree
print("Features used in the decision tree:")
print(used_features.tolist())


Features used in the decision tree:
['LOAN', 'MORTDUE', 'VALUE', 'YOJ', 'DEROG', 'DELINQ', 'CLAGE', 'NINQ', 'CLNO', 'DEBTINC', 'REASON_HomeImp', 'JOB_Office', 'JOB_Other', 'JOB_ProfExe', 'JOB_Sales', 'JOB_Self']
